In [ ]:
!pip install openbb

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import pandas as pd
import requests
import os
import numpy as np, pandas as pd
from datetime import datetime, timezone
from math import sqrt, exp, log
from scipy.stats import norm
from scipy.optimize import brentq
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401
import yfinance as yf
from openbb import obb

In [47]:
TICKER='JPM'

In [48]:
get_chains = obb.derivatives.options.chains(symbol=TICKER)

In [62]:
ALL = get_chains.to_dataframe()
ALL.rename(columns={'expiration': 'expiry'}, inplace=True)
ALL.rename(columns={'option_type':'type'}, inplace = True)

In [66]:
ASOF = datetime.now(timezone.utc).date()
ALL['asof']=pd.to_datetime(ASOF)

In [67]:
display(ALL.head(10))

,underlying_symbol,underlying_price,contract_symbol,expiry,dte,strike,type,open_interest,volume,last_trade_price,last_trade_time,bid,ask,change,change_percent,implied_volatility,in_the_money,currency,asof
0,JPM,304.4986,JPM251121P00115000,2025-11-21,6,115.0,put,330,1,0.03,2025-10-06 10:00:57-04:00,0.00,0.24,0.0,0.0,2.750003,False,USD,2025-11-15
1,JPM,304.4986,JPM251121P00120000,2025-11-21,6,120.0,put,26,1,0.04,2025-10-09 11:19:00-04:00,0.00,1.14,0.0,0.0,3.222658,False,USD,2025-11-15
2,JPM,304.4986,JPM251121P00125000,2025-11-21,6,125.0,put,64,1,0.03,2025-10-24 11:17:02-04:00,0.00,0.98,0.0,0.0,3.021487,False,USD,2025-11-15
3,JPM,304.4986,JPM251121C00130000,2025-11-21,6,130.0,call,0,2,180.50,2025-10-03 15:47:15-04:00,171.85,175.65,0.0,0.0,2.464848,True,USD,2025-11-15
4,JPM,304.4986,JPM251121P00130000,2025-11-21,6,130.0,put,15,2,0.05,2025-10-20 11:32:28-04:00,0.00,0.06,0.0,0.0,2.109380,False,USD,2025-11-15
5,JPM,304.4986,JPM251121P00135000,2025-11-21,6,135.0,put,23,3,0.01,2025-10-31 09:56:28-04:00,0.00,0.08,0.0,0.0,2.070317,False,USD,2025-11-15
6,JPM,304.4986,JPM251121P00140000,2025-11-21,6,140.0,put,39,1,0.06,2025-10-29 15:41:43-04:00,0.00,0.04,0.0,0.0,1.859376,False,USD,2025-11-15
7,JPM,304.4986,JPM251121C00145000,2025-11-21,6,145.0,call,0,2,92.70,2025-04-17 14:25:21-04:00,122.30,126.05,0.0,0.0,0.000010,True,USD,2025-11-15
8,JPM,304.4986,JPM251121P00145000,2025-11-21,6,145.0,put,75,1,0.02,2025-11-11 10:28:59-05:00,0.00,0.08,0.0,0.0,1.898438,False,USD,2025-11-15
9,JPM,304.4986,JPM251121C00150000,2025-11-21,6,150.0,call,0,1,165.67,2025-09-24 10:17:00-04:00,152.20,155.80,0.0,0.0,2.368168,True,USD,2025-11-15


In [68]:
try:
    ALL.to_parquet('/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/combined_options_data.parquet', index=False)
    print("Successfully saved combined_options_data.parquet to Google Drive.")
except Exception as e:
    print(f"Error saving file to Google Drive: {e}")
    print("Make sure you have mounted your Google Drive correctly.")

Successfully saved combined_options_data.parquet to Google Drive.


In [69]:
# Underlying historical prices (for realised vol, returns, etc.)
hist_price = yf.Ticker(TICKER).history(period="2y")  # choose horizon
hist_price_path = '/content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet'
hist_price.to_parquet(hist_price_path)
print(f"Saved underlying history to {hist_price_path}")


Saved underlying history to /content/drive/MyDrive/Colab Notebooks/VolSurf_ML/hist_price.parquet
